In [111]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/iris/Iris.csv
/kaggle/input/iris/database.sqlite


1. Kütüphaneleri İçe Aktar
2. Veriyi Tanı (EDA – Keşifsel Veri Analizi)
3. Hedef (y) ve Özellikleri (X) Ayır
4. Hedef Değişkeni Sayısallaştır
5. Train / Test Split
6. Feature Scaling
7. İlk Modeli Kur – Logistic Regression
8. Tahmin Yap
9. Model Performansını Değerlendir
10. (İkinci Aşama) Başka Model Dene

## Kütüphaneleri İçe Aktar ##

In [112]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

## Veriyi Tanı (EDA – Keşifsel Veri Analizi) ##

In [113]:
df = pd.read_csv('/kaggle/input/iris/Iris.csv')
df

,Id,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Species
0,1,5.1,3.5,1.4,0.2,Iris-setosa
1,2,4.9,3.0,1.4,0.2,Iris-setosa
2,3,4.7,3.2,1.3,0.2,Iris-setosa
3,4,4.6,3.1,1.5,0.2,Iris-setosa
4,5,5.0,3.6,1.4,0.2,Iris-setosa
...,...,...,...,...,...,...
145,146,6.7,3.0,5.2,2.3,Iris-virginica
146,147,6.3,2.5,5.0,1.9,Iris-virginica
147,148,6.5,3.0,5.2,2.0,Iris-virginica
148,149,6.2,3.4,5.4,2.3,Iris-virginica


In [114]:
df.shape

(150, 6)

In [115]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150 entries, 0 to 149
Data columns (total 6 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Id             150 non-null    int64  
 1   SepalLengthCm  150 non-null    float64
 2   SepalWidthCm   150 non-null    float64
 3   PetalLengthCm  150 non-null    float64
 4   PetalWidthCm   150 non-null    float64
 5   Species        150 non-null    object 
dtypes: float64(4), int64(1), object(1)
memory usage: 7.2+ KB


In [116]:
df.isnull().sum()

Id               0
SepalLengthCm    0
SepalWidthCm     0
PetalLengthCm    0
PetalWidthCm     0
Species          0
dtype: int64

In [117]:
df.describe()

,Id,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm
count,150.000000,150.000000,150.000000,150.000000,150.000000
mean,75.500000,5.843333,3.054000,3.758667,1.198667
std,43.445368,0.828066,0.433594,1.764420,0.763161
min,1.000000,4.300000,2.000000,1.000000,0.100000
25%,38.250000,5.100000,2.800000,1.600000,0.300000
50%,75.500000,5.800000,3.000000,4.350000,1.300000
75%,112.750000,6.400000,3.300000,5.100000,1.800000
max,150.000000,7.900000,4.400000,6.900000,2.500000


## Hedef (y) ve Özellikleri (X) Ayır ##

X (features): Modelin giriş verisi, çiçeğin ölçümleri

y (target/label): Modelin tahmin edeceği değer, çiçek türü (Species)

In [118]:
# Özellikler
X = df[['SepalLengthCm', 'SepalWidthCm', 'PetalLengthCm', 'PetalWidthCm']]

# Hedef değişken
y = df['Species']

In [119]:
df

,Id,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Species
0,1,5.1,3.5,1.4,0.2,Iris-setosa
1,2,4.9,3.0,1.4,0.2,Iris-setosa
2,3,4.7,3.2,1.3,0.2,Iris-setosa
3,4,4.6,3.1,1.5,0.2,Iris-setosa
4,5,5.0,3.6,1.4,0.2,Iris-setosa
...,...,...,...,...,...,...
145,146,6.7,3.0,5.2,2.3,Iris-virginica
146,147,6.3,2.5,5.0,1.9,Iris-virginica
147,148,6.5,3.0,5.2,2.0,Iris-virginica
148,149,6.2,3.4,5.4,2.3,Iris-virginica


In [120]:
print(X.head())

   SepalLengthCm  SepalWidthCm  PetalLengthCm  PetalWidthCm
0            5.1           3.5            1.4           0.2
1            4.9           3.0            1.4           0.2
2            4.7           3.2            1.3           0.2
3            4.6           3.1            1.5           0.2
4            5.0           3.6            1.4           0.2


In [121]:
print(y[:5])

0    Iris-setosa
1    Iris-setosa
2    Iris-setosa
3    Iris-setosa
4    Iris-setosa
Name: Species, dtype: object


## Hedef Değişkeni Sayısallaştır ##

Label Encoding

In [122]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
y_encoded = le.fit_transform(y)

print(list(y[:10]))        # orijinal

['Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa']


In [123]:
print(list(y_encoded[:10])) # sayısallaştırılmış

[np.int64(0), np.int64(0), np.int64(0), np.int64(0), np.int64(0), np.int64(0), np.int64(0), np.int64(0), np.int64(0), np.int64(0)]


## Train / Test Split ##

Train: Modelin öğrendiği veri

Test: Modelin daha önce görmediği veri ile değerlendirme

In [124]:
from sklearn.model_selection import train_test_split

# X = özellikler, y_encoded = sayısal hedef
X_train, X_test, y_train, y_test = train_test_split(
    X, y_encoded, test_size=0.2, random_state=42, stratify=y_encoded
)

In [125]:
print(X_train.shape, X_test.shape)
X_train

(120, 4) (30, 4)


,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm
8,4.4,2.9,1.4,0.2
106,4.9,2.5,4.5,1.7
76,6.8,2.8,4.8,1.4
9,4.9,3.1,1.5,0.1
89,5.5,2.5,4.0,1.3
...,...,...,...,...
37,4.9,3.1,1.5,0.1
2,4.7,3.2,1.3,0.2
33,5.5,4.2,1.4,0.2
52,6.9,3.1,4.9,1.5


In [126]:
X_test

,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm
38,4.4,3.0,1.3,0.2
127,6.1,3.0,4.9,1.8
57,4.9,2.4,3.3,1.0
93,5.0,2.3,3.3,1.0
42,4.4,3.2,1.3,0.2
56,6.3,3.3,4.7,1.6
22,4.6,3.6,1.0,0.2
20,5.4,3.4,1.7,0.2
147,6.5,3.0,5.2,2.0
84,5.4,3.0,4.5,1.5


In [127]:
print(y_train.shape, y_test.shape)
y_train

(120,) (30,)


array([0, 2, 1, 0, 1, 2, 1, 2, 2, 2, 2, 1, 1, 1, 1, 0, 0, 2, 2, 0, 1, 0,
       2, 0, 1, 2, 2, 0, 2, 0, 0, 1, 1, 0, 2, 2, 1, 1, 2, 1, 0, 1, 0, 2,
       0, 0, 2, 0, 0, 0, 0, 1, 2, 1, 0, 2, 1, 2, 0, 2, 0, 1, 2, 0, 1, 1,
       2, 1, 1, 2, 0, 0, 0, 2, 1, 2, 1, 2, 2, 1, 0, 2, 1, 0, 2, 0, 2, 1,
       1, 0, 1, 2, 0, 0, 2, 2, 2, 1, 2, 0, 2, 1, 2, 2, 0, 1, 1, 1, 1, 1,
       0, 2, 1, 1, 0, 0, 0, 0, 1, 0])

In [128]:
y_test

array([0, 2, 1, 1, 0, 1, 0, 0, 2, 1, 2, 2, 2, 1, 0, 0, 0, 1, 1, 2, 0, 2,
       1, 2, 2, 1, 1, 0, 2, 0])

## Feature Scaling ##

Eğer ölçeklendirme yapılmazsa, özellikler farklı ağırlıkta algılanır.

Feature Scaling: Tüm özellikleri benzer ölçeğe getirme işlemi

In [129]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

# Sadece X_train üzerinde fit et, X_test’i transform et
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [130]:
X_train

,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm
8,4.4,2.9,1.4,0.2
106,4.9,2.5,4.5,1.7
76,6.8,2.8,4.8,1.4
9,4.9,3.1,1.5,0.1
89,5.5,2.5,4.0,1.3
...,...,...,...,...
37,4.9,3.1,1.5,0.1
2,4.7,3.2,1.3,0.2
33,5.5,4.2,1.4,0.2
52,6.9,3.1,4.9,1.5


In [131]:
X_train_scaled

array([[-1.72156775, -0.32483982, -1.34703555, -1.32016847],
       [-1.12449223, -1.22612948,  0.41429037,  0.65186742],
       [ 1.14439475, -0.55016223,  0.58474127,  0.25746024],
       [-1.12449223,  0.12580502, -1.29021859, -1.45163753],
       [-0.40800161, -1.22612948,  0.13020555,  0.12599118],
       [ 0.54731923, -1.22612948,  0.6983752 ,  0.91480554],
       [-0.2885865 , -0.77548465,  0.24383948,  0.12599118],
       [ 0.54731923, -0.55016223,  0.75519216,  0.3889293 ],
       [ 2.21913069, -0.0995174 ,  1.32336182,  1.44068178],
       [ 2.21913069,  1.70306192,  1.66426361,  1.30921272],
       [ 2.09971558, -0.0995174 ,  1.60744665,  1.17774366],
       [ 0.18907392, -0.32483982,  0.41429037,  0.3889293 ],
       [-1.00507713, -2.35274155, -0.15387928, -0.268416  ],
       [-0.04975629, -0.77548465,  0.18702251, -0.268416  ],
       [-0.04975629, -1.00080706,  0.13020555, -0.00547788],
       [-1.36332244,  0.35112743, -1.23340162, -1.32016847],
       [-0.88566202,  1.

## İlk Modeli Kur – Logistic Regression ##

In [132]:
from sklearn.linear_model import LogisticRegression

In [133]:
# Modeli tanımla
model = LogisticRegression(
    max_iter=200,
    random_state=42
)

# Modeli Eğit (Fit)
model.fit(X_train_scaled, y_train)

# Tahmin Yap
y_pred = model.predict(X_test_scaled)

### Model Performansını Değerlendir ###

In [134]:
# 1. Accuracy (Doğruluk)
from sklearn.metrics import accuracy_score

accuracy = accuracy_score(y_test, y_pred)
print("Accuracy Lc:", accuracy)

Accuracy Lc: 0.9333333333333333


In [135]:
# 2. Confusion Matrix
from sklearn.metrics import confusion_matrix

cm = confusion_matrix(y_test, y_pred)
print('LogisticConfusion')
print(cm)

LogisticConfusion
[[10  0  0]
 [ 0  9  1]
 [ 0  1  9]]


In [136]:
# 3. Classification Report
from sklearn.metrics import classification_report
print('Lc Classification')
print(classification_report(y_test, y_pred))

Lc Classification
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        10
           1       0.90      0.90      0.90        10
           2       0.90      0.90      0.90        10

    accuracy                           0.93        30
   macro avg       0.93      0.93      0.93        30
weighted avg       0.93      0.93      0.93        30



### (İkinci Aşama) Başka Model Dene ###

## K-Nearest Neighbors (KNN) ##

1. Feature Scaling
2. K Değerini Seç (n_neighbors)
3. Modeli Eğit (fit)
4. Tahmin Yap (predict)
5. Modeli Değerlendir

In [137]:
from sklearn.neighbors import KNeighborsClassifier

knn = KNeighborsClassifier(n_neighbors=5)

# Modeli Eğit (fit)
knn.fit(X_train_scaled, y_train)

# Tahmin Yap (predict)
y_pred_knn = knn.predict(X_test_scaled)

In [138]:
# Modeli Değerlendir
# 1. Accuracy

from sklearn.metrics import accuracy_score
accuracy_score(y_test, y_pred_knn)

print("Accuracy Knn:", accuracy)

Accuracy Knn: 0.9333333333333333


In [139]:
# 2. Confusion Matrix
from sklearn.metrics import confusion_matrix
cm2 = confusion_matrix(y_test, y_pred_knn)
print('KnnConfusion')
print(cm2)

KnnConfusion
[[10  0  0]
 [ 0 10  0]
 [ 0  2  8]]


In [140]:
# 3. Classification Report
from sklearn.metrics import classification_report
print('Knn Classification')
print(classification_report(y_test, y_pred_knn))

Knn Classification
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        10
           1       0.83      1.00      0.91        10
           2       1.00      0.80      0.89        10

    accuracy                           0.93        30
   macro avg       0.94      0.93      0.93        30
weighted avg       0.94      0.93      0.93        30



### Logistic Regression ne zaman daha iyi? ###

Veri büyükse

Modelin yorumlanabilir olması gerekiyorsa

Üretim ortamında hız önemliyse

Overfitting riski düşük olsun istiyorsan

### KNN ne zaman daha iyi? ###

Veri küçükse

Sınıflar doğal olarak kümeleniyorsa

Karar sınırları doğrusal değilse